In [ ]:
%tensorflow_version 2.x

In [ ]:
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
2.8835851219999995
GPU (s):
0.10396243099999936
GPU speedup over CPU: 27x


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import csv

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print(tf.version.VERSION)

2.3.0


In [ ]:
FEC_base = keras.models.load_model("./drive/My Drive/Models/FEC/FEC_base")
FEC_emotion = keras.models.load_model("./drive/My Drive/Models/FECNet_fer_no_outliers_19_8.h5")
FEC_sleep = keras.models.load_model("./drive/My Drive/Models/FECNet_sleep_19_8.h5")

In [ ]:
emotion_branch = FEC_emotion.layers[-1]
emotion_branch.summary()

Model: "Emotion_Classifier"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 16)]              0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               8704      
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 2052      
Total params: 10,756
Trainable params: 0
Non-trainable params: 10,756
_________________________________________________________________


In [ ]:
FEC_sleep.summary()

Model: "FECNet_Classifier"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
FEC_base (Functional)        (None, 16)                5879264   
_________________________________________________________________
dense (Dense)                (None, 512)               8704      
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 1026      
Total params: 5,888,994
Trainable params: 9,730
Non-trainable params: 5,879,264
_________________________________________________________________


In [ ]:
sleep_branch = keras.models.Sequential(layers=[
                                               layers.Input((16)),
                                               FEC_sleep.layers[1],
                                               FEC_sleep.layers[2],
], name="Sleep_branch")
sleep_branch.summary()

Model: "Sleep_branch"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               8704      
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 1026      
Total params: 9,730
Trainable params: 9,730
Non-trainable params: 0
_________________________________________________________________


In [ ]:
x = FEC_base.output
y = sleep_branch(x)
z = emotion_branch(x)
x = layers.Concatenate()([y,z])
FEC_emotion_sleep = keras.Model(inputs=FEC_base.input, outputs=[y,z], name="FEC_Emotion_Sleep")
FEC_emotion_sleep.summary()

Model: "FEC_Emotion_Sleep"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
FaceNet_base (Functional)       (None, 8, 8, 896)    3406032     input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 8, 8, 512)    458752      FaceNet_base[0][0]               
__________________________________________________________________________________________________
dense1_block1_0_bn (BatchNormal (None, 8, 8, 512)    2048        conv2d[0][0]                     
__________________________________________________________________________________

In [ ]:
FEC_emotion_sleep.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
FEC_emotion_sleep.save("./drive/My Drive/Models/FEC_Emotion_Sleep_20_8.h5")

In [ ]:
FEC_emotion.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
FEC_sleep.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
FEC_emotion.save("./drive/My Drive/Models/FEC_Emotion_20_8")
FEC_sleep.save("./drive/My Drive/Models/FEC_Sleep_20_8")
# FEC_emotion_sleep.save("./drive/My Drive/Models/FEC_Emotion_Sleep_20_8")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ./drive/My Drive/Models/FEC_Emotion_20_8/assets
INFO:tensorflow:Assets written to: ./drive/My Drive/Models/FEC_Sleep_20_8/assets
INFO:tensorflow:Assets written to: ./drive/My Drive/Models/FEC_Emotion_Sleep_20_8/assets


In [ ]:
FEC_emotion.save("./drive/My Drive/Models/FEC_Emotion_20_8.h5")
FEC_sleep.save("./drive/My Drive/Models/FEC_Sleep_20_8.h5")